In [5]:
from pymongo import ASCENDING, GEOSPHERE, MongoClient
import pandas as pd
from alive_progress import alive_bar
from shapely.geometry import Point, mapping
import shapely
import json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import geopandas as gpd

In [6]:
client = MongoClient('mongodb://localhost:27017/')

In [7]:
db = client.osmDataDB

In [8]:
coll_areas = db.areas

In [9]:
coll_relations = db.relations

In [10]:
coll_nextBikeRaw = db.nextBikeRaw

In [94]:
coll_nextBikeRaw.create_index([("area_osm_id", ASCENDING)])
coll_nextBikeRaw.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [11]:
def check_duplicate_areas():
    areas = coll_areas.find()
    for area in areas:
        query = { 'geometry': { '$geoIntersects': { '$geometry': area['geometry'] } } }
        areas_prime = coll_areas.find(query)
        for area_prime in areas_prime:
            if area['osm_id'] != area_prime['osm_id']:
                print(area['relation_name'], area['osm_id'], area_prime['relation_name'], area_prime['osm_id'])

In [22]:
check_duplicate_areas()

Munich, Germany 62428 München-Land, Landkreis München, Bayern, Deutschland 3146373
München-Land, Landkreis München, Bayern, Deutschland 3146373 Munich, Germany 62428


In [29]:
def clean_orphan_records():
    osm_ids = [doc['osm_id'] for doc in coll_areas.find()]
    d = coll_relations.delete_many({ 'parent_osm_id': { '$nin': osm_ids } })
    print(d.deleted_count, " documents deleted") 

In [30]:
clean_orphan_records()

0  documents deleted


In [27]:
data_directory_path = '/mnt/Storage/PwrDatasets/BicycleSharing/stations'

In [28]:
def get_area_id(point: Point):
    # print(point)
    query = { 'geometry': { '$geoIntersects': { '$geometry': mapping(point) } } }
    area = coll_areas.find_one(query)
    try:
        return area['osm_id']
    except:
        print('missing area', point)
        # print(query)
        return None

In [29]:
def get_point(lat, lon):
    return Point(lon, lat)

In [30]:
osm_id = get_area_id(get_point(51.217820000000000000,4.420650000000000000))
print(osm_id)

59518


In [119]:
def iterate_bulk_csv():
    path = data_directory_path + '/bulk'
    files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    dfs = []
    fields = ['lat', 'lon', 'station_name', 'temporary']
    for f in files:
        # print(pd.read_csv(f, skipinitialspace=True, usecols=fields)['name'])
        df = pd.read_csv(f, skipinitialspace=True, usecols=fields)
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df.temporary != True]
    # with alive_bar(len(df), title=f"Saving stations to MongoDB") as bar:
    for index, row in tqdm(df.iterrows(), total=len(df)):
        # print(row)
        pt = get_point(row.lat, row.lon)
        if (row.lat is int or row.lon is int):
            print('INTEGER', pt)
            continue
        area_id = get_area_id(pt)
        if area_id:
            record = {
                'area_osm_id': area_id,
                'name': row.station_name,
                'geometry': mapping(pt)
            }
            # print(record['name'])
            coll_nextBikeRaw.replace_one({'geometry':record['geometry']}, record, True)
            # bar()

In [120]:
iterate_bulk_csv()

 24%|██▍       | 2694/11339 [00:15<01:20, 107.59it/s]missing area POINT (-3.0583068367 55.94396074180001)
missing area POINT (-3.06675391622 55.94388031689999)
missing area POINT (-3.07310463368 55.9319804826)
 80%|███████▉  | 9021/11339 [02:23<00:20, 112.77it/s]missing area POINT (16.88657104969025 52.35371753650325)
missing area POINT (16.882107 52.352165)
missing area POINT (16.88072 52.348828)
missing area POINT (16.882574 52.34329)
missing area POINT (16.892548 52.344134)
missing area POINT (16.861076 52.349716)
missing area POINT (16.88953 52.33300999999999)
missing area POINT (16.89727 52.34819)
missing area POINT (16.86261 52.3427)
missing area POINT (16.87815 52.32742)
missing area POINT (16.87191 52.33963)
missing area POINT (16.80782 52.36555)
missing area POINT (16.81169 52.37413)
 98%|█████████▊| 11129/11339 [02:50<00:02, 100.06it/s]missing area POINT (21.01796150207519 52.06681812814481)
missing area POINT (21.02705419063568 52.07282949167776)
missing area POINT (21.05214

In [63]:
pd.read_csv('/mnt/Storage/PwrDatasets/BicycleSharing/stations/bulk/madrid.csv')

,id,name,lat,lon,bikes,spaces,installed,locked,temporary,open_or_total_docks,bonus,ticket,xd
1,1,Puerta del Sol A,40.417214,-3.701834,28,1,True,False,False,30,NaN,NaN,NaN
2,2,Puerta del Sol B,40.417313,-3.701603,14,16,True,False,False,30,NaN,NaN,NaN
3,3,Miguel Moya,40.420589,-3.705842,8,15,True,False,False,24,NaN,NaN,NaN
4,4,Plaza Conde Suchil,40.430294,-3.706917,6,7,True,False,False,18,NaN,NaN,NaN
5,5,Malasaña,40.428552,-3.702587,12,11,True,False,False,24,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,265,INEF,40.438960,-3.729970,14,6,True,False,False,24,NaN,NaN,NaN
266,266,Ciudad Universitaria 1,40.443750,-3.726990,5,16,True,False,False,24,NaN,NaN,NaN
267,267,Ciudad Universitaria 2,40.443420,-3.726930,15,9,True,False,False,24,NaN,NaN,NaN
268,268,Facultad Biología,40.449120,-3.727310,15,6,True,False,False,24,NaN,NaN,NaN


In [67]:
dijon = json.load(open('/mnt/Storage/PwrDatasets/BicycleSharing/stations/dijon.json'))
lausanne_bern = json.load(open('/mnt/Storage/PwrDatasets/BicycleSharing/stations/lausanne_bern.json'))
malmo = json.load(open('/mnt/Storage/PwrDatasets/BicycleSharing/stations/malmo.json'))

In [71]:
dijon_parsed = [{
    'name': x['infos']['nom'],
    'lat': float(x['infos']['lat']),
    'lon': float(x['infos']['lon'])
} for x in dijon]

In [78]:
lausanne_bern = [{
    'name': x['id'],
    'lat': float(x['lat']),
    'lon': float(x['lon'])
} for x in lausanne_bern]

In [80]:
malmo = [{
    'name': x['name'],
    'lat': float(x['lat']),
    'lon': float(x['lon'])
} for x in malmo]

In [121]:
stations_json = dijon_parsed + lausanne_bern + malmo
for row in tqdm(stations_json):
    # print(row)
    pt = get_point(row['lat'], row['lon'])
    area_id = get_area_id(pt)
    if area_id:
        record = {
            'area_osm_id': area_id,
            'name': row['name'],
            'geometry': mapping(pt)
        }
        coll_nextBikeRaw.replace_one({'geometry':record['geometry']}, record, True)

  9%|▉         | 64/684 [00:00<00:09, 62.68it/s]missing area POINT (8.957940000000001 46.0105808)
missing area POINT (6.23856 46.39294)
missing area POINT (6.5655126 46.5224054)
missing area POINT (8.9192 45.9242)
missing area POINT (8.927351 45.973369)
 12%|█▏        | 84/684 [00:01<00:10, 57.84it/s]missing area POINT (8.917052 45.958424)
missing area POINT (6.579268 46.526079)
missing area POINT (8.959651300000001 46.0166117)
missing area POINT (8.943799 45.991081)
 15%|█▌        | 106/684 [00:01<00:08, 65.28it/s]missing area POINT (8.958952 46.013865)
missing area POINT (8.962901 46.021942)
missing area POINT (6.225766 46.383066)
missing area POINT (7.15074 46.77787)
missing area POINT (7.377833 46.235077)
missing area POINT (8.945138999999999 45.999094)
missing area POINT (7.147493 46.79816)
 18%|█▊        | 120/684 [00:02<00:16, 34.36it/s]missing area POINT (6.19271 46.31526)
missing area POINT (8.955118000000001 46.012252)
missing area POINT (7.11461 46.81288)
 20%|█▉        | 13

In [149]:
records = [r for r in coll_nextBikeRaw.find()]

In [156]:
for r in records:
    r['geometry'] = shapely.geometry.shape(r['geometry'])
    del r['_id']

In [157]:
df = gpd.GeoDataFrame([r for r in records])

In [158]:
df

,area_osm_id,name,geometry
0,1390623,Headford Road,POINT (-9.05069 53.27835)
1,1390623,Cathedral,POINT (-9.05724 53.27584)
2,1390623,Wood Quay,POINT (-9.05308 53.27638)
3,1390623,County Hall,POINT (-9.04800 53.27600)
4,1390623,Galway City Hall,POINT (-9.04300 53.27600)
...,...,...,...
11821,10663667,Helsingborgsgatan,POINT (13.01106 55.59574)
11822,10663667,Moriskan,POINT (13.01159 55.59275)
11823,10663667,Falsterboplan,POINT (13.01452 55.59045)
11824,10663667,Spiggans plats,POINT (13.01976 55.60170)


In [159]:
source_json = df.to_json(na='drop')
with open('nextbike.geojson', 'w', encoding='utf-8') as source:
    source.write(source_json)

In [25]:
areas_records = [r for r in coll_areas.find()]

In [26]:
for r in areas_records:
    r['geometry'] = shapely.geometry.shape(r['geometry'])
    del r['_id']

In [27]:
df_areas = gpd.GeoDataFrame([r for r in areas_records])
df_areas

,osm_id,relation_name,administration_name,geometry
0,2805691,"Wrocław, Poland","Wrocław, powiat wrocławski, województwo dolnoś...","POLYGON ((16.80734 51.13895, 16.80859 51.13887..."
1,175342,"Greater London, UK","Greater London, England, United Kingdom","POLYGON ((-0.51038 51.46809, -0.51036 51.46795..."
2,109166,"Vienna, Austria","Wien, Österreich","POLYGON ((16.18183 48.17112, 16.18190 48.17103..."
3,59518,"Antwerp, Belgium","Antwerpen, Vlaanderen, België / Belgique / Bel...","POLYGON ((4.21758 51.37389, 4.21826 51.37221, ..."
4,2628520,"Greater Nicosia, Nicosia District, Cyprus","Λευκωσία - Lefkoşa, Λευκωσία, Κύπρος, Κύπρος -...","POLYGON ((33.26978 35.13287, 33.27003 35.13262..."
...,...,...,...,...
61,44880,"Torino, Piemonte, Italia","Torino, Piemonte, Italia","POLYGON ((6.62727 45.10680, 6.62728 45.10622, ..."
62,1685958,"Dijon, Côte-d'Or, Bourgogne-Franche-Comté, Fra...","Dijon, Côte-d'Or, Bourgogne-Franche-Comté, Fra...","POLYGON ((4.56872 47.32121, 4.56929 47.32079, ..."
63,1690227,"Zürich, Schweiz/Suisse/Svizzera/Svizra","Zürich, Schweiz/Suisse/Svizzera/Svizra","POLYGON ((8.35768 47.50253, 8.35783 47.50247, ..."
64,1686344,"Bern/Berne, Schweiz/Suisse/Svizzera/Svizra","Bern/Berne, Schweiz/Suisse/Svizzera/Svizra","MULTIPOLYGON (((6.86148 47.16563, 6.86152 47.1..."


In [28]:
source_json = df_areas.to_json(na='drop')
with open('areas.geojson', 'w', encoding='utf-8') as source:
    source.write(source_json)